# Проверка корректности работы градиентного спуска

Ваша задача в компании это построить модель на основе методов глубокого обучения по детектированию аномалий в платежах совершаемых пользователями с использованием мобильных устройств, поскольку некоторые платежы могут совершаться мошенниками или хакерами.

Но обратное распространение ошибки довольно сложно реализовать и иногда, при реализации, алгоритм имеет ошибки. Ваш руководитель хочет, чтобы вы реализовали метод проверки корректности работы обратного распространения ошибки. 

`Данный материал опирается и использует материалы курса Deep Learning от организации deeplearning.ai`
 
 Ссылка на осной курс (для желающих получить сертификаты): https://www.coursera.org/specializations/deep-learning

## 1 - Пакеты/Библиотеки ##

Первоначально необходимо запустить ячейку ниже, чтобы импортировать все пакеты, которые вам понадобятся во время лабораторной работы.
- [numpy](www.numpy.org) является основным пакетом для научных вычислений в Python.
- gc_utils предоставляет различные полезные функции, используемые в данной лабораторной работе.

In [1]:
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 1) Как проверить работы градиентного спуска?

В алгоритме обратного распространения ошибки вычисляются градиенты  $\frac{\partial J}{\partial \theta}$, где $\theta$ соответствует параметрам модели. $J$ вычислялось с использованием прямого распросранения и и функции потерь.

Поскольку прямое распространение и функцию потерь относительно легко реализовать, то можно быть увереным практически на 100% что вычисление $J$ корректно. В связи с этом можно использовать вычисленное значение $J$ для проверки и вычисления $\frac{\partial J}{\partial \theta}$. 

Определение производной (или градиентов):
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

В рамках данной лабораторной работы вы узнаете как:

- $\frac{\partial J}{\partial \theta}$ - убедиться в том, что это выражение вычисляется корректно. 
- Вычислить $J(\theta + \varepsilon)$ и $J(\theta - \varepsilon)$ (при условии что $\theta$ действительное число), при условии уверенности в том, что верно вычисленно $J$. 

## 2) Проверка градиента в одномерном измерении

Рассмотрим одномерную линейную функцию $J(\theta) = \theta x$. Модель содержит единственный параметр $\theta$, и принимает на вход $x$.

Необходимо вычислить  $J(.)$ и её производную $\frac{\partial J}{\partial \theta}$. Затем необходимо использовать проверку градиента для того, чтобы убедится в корректности расчёта $J$. 

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center> <u> **Figure 1** </u>: **1D linear model**<br> </center></caption>

На диаграмме выше показано ключевые шаги вычисления: первый шаг начинается с $x$, затем вычисляется функция функция потерь. Затем вычисляется производная $\frac{\partial J}{\partial \theta}$ ("backward propagation"). 

**Упражнение**: Реализовать "прямое распространение" и "обратное распространение" для этой простой функции. Т.е., вычислить и $J(.)$ и её производную по $\theta$, с использованием двух раздельных функций. 

In [2]:
# ОЦЕНИВАЕМОЕ: forward_propagation

def forward_propagation(x, theta):
    """
    Реализация функции линейного прямого распространения (вычисление потерь) (J(theta) = theta * x)
    
    Arguments:
    x -- действительное число
    theta -- параметр thetha, также число
    
    Returns:
    J -- значение функции потерь J, вычисленное с использованием формулы J(theta) = theta * x
    """
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (прибл. 1 строка кода)
    J = np.dot(theta, x)
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    return J

In [3]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


**Ожидаемый результат**:

<table style=>
    <tr>
        <td>  ** J **  </td>
        <td> 8 </td>
    </tr>
</table>

**Упражнение**: Реализовать обратное распространение. Для этого вычислить производную $J(\theta) = \theta x$ по $\theta$. Сохранить вычисления в переменную $dtheta = \frac { \partial J }{ \partial \theta} = x$.

In [4]:
# ОЦЕНИВАЕМОЕ: backward_propagation

def backward_propagation(x, theta):
    """
    Вычисление производной по функции J по параметру thets.
    
    Arguments:
    x -- действительное число
    theta -- параметр thetha, также число
    
    Returns:
    dtheta -- градиент по theta
    """
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (прибл. 1 строка кода)
    dtheta = x
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    return dtheta

In [5]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


**Ожидаемый результат**:

<table>
    <tr>
        <td>  ** dtheta **  </td>
        <td> 2 </td>
    </tr>
</table>

**Упражнение**: Показать, что `backward_propagation()` функция корректно вычисляет градиент $\frac{\partial J}{\partial \theta}$.

**Инструкция**:
- Вычислите "gradapprox" с использованием формулы (1) и небольшого числа $\varepsilon$. Следующие шаги:
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- Затем вычислите градиент с использованием обратного распространения и сохраните результаты в переменную "grad"
- В итоге вычислите относительное различие между "gradapprox" и "grad" с использованием следующей формулы:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$
3 Steps для вычисления этой формулы
   - 1'. вычислить числитель с использованием np.linalg.norm(...)
   - 2'. вычислить знаменатель с использованием np.linalg.norm(...).
   - 3'. разделить их.
- Если различие небольшое (скажем $10^{-7}$), то можно с большой долей уверенности утверждать что градиент вычислен корректно. С другой стороны, большое различие показывает, что в вычислении обратного распространения закралась ошибка.


In [6]:
# ОЦЕНИВАЕМОЕ: gradient_check

def gradient_check(x, theta, epsilon = 1e-7):
    """
    Реализация обратного распространения.
    
    Arguments:
    x -- действительное число
    theta -- параметр thetha, также число
    epsilon -- небольшой сдвиг между координатами
    
    Returns:
    difference -- различие между апроксимацией градиента и обратным распространением
    """
    # Вычислите gradapprox
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (прибл. 5 строк кода)
    thetaplus = theta + epsilon         
    thetaminus = theta - epsilon                              
    J_plus = np.dot(thetaplus, x)                                 
    J_minus = np.dot(thetaminus, x)
    gradapprox = (J_plus - J_minus)/(2*epsilon)
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    
    # Вычислите grad
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (прибл. 1 строка кода)
    grad = x
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###

    # Проверьте gradapprox
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (прибл. 1-3 строки кода)
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator/denominator
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    if difference < 1e-7:
        print ("Вычисление градиента корректено!")
    else:
        print ("В вычислении градиента ошибка!")
    
    return difference

In [7]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

Вычисление градиента корректено!
difference = 2.919335883291695e-10


**Ожидаемый результат**:
Вычисление градиента корректено!
<table>
    <tr>
        <td>  ** difference **  </td>
        <td> 2.919335883291695e-10 </td>
    </tr>
</table>

При обучении нейронной сети обычно $\theta$ содержит несколько матриц $W^{[l]}$ и векторов смещений $b^{[l]}$. В следующей части лабораторной работы необходимо реализовать проверку работы обратного распространения для N-слойной нейронной сети.

## 3) Проверка градиента в N-мерном измерении

На следующем изображении представлена диаграмма описывающая прямое и обратное распространение для модели детектирования аномалий.

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center> <u> **Figure 2** </u>: Глубокая нейронная сеть<br>*LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID*</center></caption>

In [8]:
def forward_propagation_n(X, Y, parameters):
    """
    Реализация прямого распространения как на Рисунке 3.
    
    Arguments:
    X -- обучающий набор примеров m
    Y -- метки для m примеров 
    parameters -- python словарь, содержащий параметры "W1", "b1", "W2", "b2", "W3", "b3":
                    W1 -- матрица весов 1 слоя (5, 4)
                    b1 -- вектор отклонений 1 слоя (5, 1)
                    W2 -- матрица весов 2 слоя (3, 5)
                    b2 -- вектор отклонений 2 слоя (3, 1)
                    W3 -- матрица весов 3 слоя (1, 3)
                    b3 -- вектор отклонений 3 слоя (1, 1)
    
    Returns:
    cost -- потери
    """
    
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Потери
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [9]:
def backward_propagation_n(X, Y, cache):
    """
    Реализация обратного распространения как на Рисунке 2.
    
    Arguments:
    X -- входные данные (input size, 1)
    Y -- метка
    cache -- кэш входных данных forward_propagation_n()
    
    Returns:
    gradients -- словарь градиентов для каждой переменной.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) 
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

Допустим вы получили некоторые результат работы вашей модели на тестовой выборке, но вы не до конца уверены в корректности работы данной модели. Давайте проверим работу обратного распространения ошибки.

Как в 1) и 2), необходимо сравнить "gradapprox" с градиентом вычисленным по формуле обратного распросранения ошибки:

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

Тем не менее, $\theta$ уже не скаляр. Это словарь "parameters". Для вас реализована функция "`dictionary_to_vector ()`. Она преобразует словарь "parameters" в Вектор под названием "values", полученный путем преобразования всех параметров (W1, b1, W2, b2, W3, b3) в векторы и объединения их.

Обратная функция "`vector_to_dictionary`" которая делает обратное преобразование в словарь "parameters".

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center> <u> **Figure 2** </u>: **dictionary_to_vector() and vector_to_dictionary()**<br> You will need these functions in gradient_check_n()</center></caption>

Также в лабораторной работе для вас уже преобразовали словарь "gradients" в вектор "grad" с помощью функции gradients_to_vector().

**Упражнение**: Реализовать gradient_check_n().

**Инструкция**: Вот псевдокод, который поможет вам реализовать проверку градиента.

For each i in num_parameters:
- Вычислите `J_plus[i]`:
    1. Набор $\theta^{+}$ скопировать в `np.copy(parameters_values)`
    2. В набор $\theta^{+}_i$ добавить $\theta^{+}_i + \varepsilon$
    3. Вычислить $J^{+}_i$ с использованием `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- Вычислить `J_minus[i]`: делать точно такие же шаги, только для $\theta^{-}$
- Вычислить $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

Таким образом, вы получите вектор gradapprox, где gradapprox[i]  аппроксимация градиента относительно `parameter_values[i]`. Теперь вы можете сравнить вектор gradapprox с вектором градиентов из обратного распространения. Вычислите:
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

In [15]:
# ОЦЕНИВАЕМОЕ: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    """
    Проверка вычисления градиента backward_propagation_n по forward_propagation_n
    
    Arguments:
    parameters -- python словарь, содержащий параметры "W1", "b1", "W2", "b2", "W3", "b3":
    grad -- выход backward_propagation_n, содержит градиенты по всем параметрам. 
    x -- взодная матрица признаков (input size, 1)
    y -- метка 
    epsilon -- небольшое значение сдвига, используемое в формуле (1)
    
    Returns:
    difference -- разиличие между градиентами, расчитанное по формуле (2)
    """
    
    # Настройка переменных
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    # Вычисление gradapprox
    for i in range(num_parameters):
        # Вычисление J_plus[i]. Вход: "parameters_values, epsilon". Выход = "J_plus[i]".
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (approx. 3 lines)
        thetaplus = np.copy(parameters_values)                                        # Шаг 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon                                   # Шаг 2
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))   # Шаг 3
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
        # Вычисление J_minus[i]. Вход: "parameters_values, epsilon". Выход = "J_minus[i]".
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (approx. 3 lines)
        thetaminus = np.copy(parameters_values)                                       # Шаг 1
        thetaminus[i][0] = thetaminus[i][0] - epsilon                                 # Шаг 2        
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus)) # Шаг 3
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
        
        # Вычисление gradapprox[i]
        ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (approx. 1 line)
        gradapprox[i] = (J_plus[i] - J_minus[i])/(2*epsilon)
        ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###
    
    # Сравнение gradapprox с обратным распространением.
    ### НАЧАЛО ВАШЕГО КОД ЗДЕСЬ ### (approx. 1 line)
    numerator = np.linalg.norm(grad - gradapprox)                                           # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                                         # Step 2'
    difference = numerator/denominator                                        # Step 3'
    ### ОКОНЧАНИЕ ВАШЕГО КОД ЗДЕСЬ ###

    if difference > 1e-7:
        print ("\033[91m" + "Ошибка в вычисление обратного распространения! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Обратное распространение ошибки работает корректно! difference = " + str(difference) + "\033[0m")
    
    return difference

In [14]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

Ошибка в вычисление обратного распространения! difference = 1.1890913023330276e-07


**Ожидаемый результат**:

<table>
    <tr>
        <td>  ** Ошибка в вычисление обратного распространения!**  </td>
        <td> difference = 1.1890913023330276e-07 </td>
    </tr>
</table>

Похоже, что в коде backward_propagation_n, который вам был предоставлен, были ошибки! Хорошо, что вы реализовали проверку градиента. Вернитесь к "backward_propagation" и попробуйте найти/исправить ошибки *(подсказка: проверьте dW2 и db1)*. Повторите проверку градиента, когда вы решите, что исправили весь код. Помните, что вам нужно будет повторно выполнить ячейку, определяющую `backward_propagation_n ()`, если вы измените код.

**Заметка** 
- Проверка градиента работает медленно! Аппроксимация градиента с помощью $\frac{\partial J}{\partial \theta} \approx  \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$ вычислительно дорого. По этой причине нет смысла выполнять проверку градиента на каждой итерации во время обучения. Это можно сделать всего несколько раз, чтобы проверить правильность расчётов градиента.
- Проверка градиента, по крайней мере, как вы ее реализовали, не работает с dropout. Алгоритм проверки градиента обычно запускается без dropout, чтобы убедиться, что backprop правильный, а затем добавляют dropout.

<font color='blue'>
Что необходимо помнить:

- Проверка градиента - это оценка близости между градиентами от обратного распространения и численной аппроксимацией градиента (вычисленной с использованием прямого распространения).
- Проверка градиента выполняется медленно, поэтому мы не запускаем ее в каждой итерации обучения. Обычно вы запускаете его только для того, чтобы убедиться в правильности кода, а затем отключаете и используете backprop для реального процесса обучения.